In [1]:
import time
import os
from parsingFunctions import parseXML
from parsingFunctions import findPMIDInXML
import pandas as pd
pd.set_option('display.max_rows', 500)

In [2]:
XML_dir = input()

 E:\sciAbstractsProject_XMLs\not_parsed_first_round\


In [3]:
DF_output = input()

  E:\sciAbstractsProject_parsedDFs\not_parsed_first_round.csv


In [4]:
XML_dir_content = os.listdir(XML_dir)

In [5]:
XML_numbers = []
XML_file = []
        
for XML in range(len(XML_dir_content)):
    XML_numbers.append(int(XML_dir_content[XML][:-4]))
        
XML_numbers.sort()

for number in range(len(XML_numbers)):
    XML_file.append(str(XML_numbers[number]) + ".txt")

In [6]:
parsed_XML_output = []
not_parsed_XML_files = []
current_parsing_XML = ''

start = time.time()
parseXML(XML_dir, XML_file, parsed_XML_output, not_parsed_XML_files, current_parsing_XML, DF_output)
print("Parsing time: " + str(round(time.time()-start,2)) + " seconds")

451400.txt not parsed
2721000.txt not parsed
3843600.txt not parsed
4216400.txt not parsed
4801600.txt not parsed
5370800.txt not parsed
6472200.txt not parsed
6474800.txt not parsed
6510800.txt not parsed
8436200.txt not parsed
8829000.txt not parsed
8833200.txt not parsed
8904600.txt not parsed
8939200.txt not parsed
8947600.txt not parsed
8954600.txt not parsed
9070400.txt not parsed
9108000.txt not parsed
10135800.txt not parsed
10896800.txt not parsed
Parsing time: 0.63 seconds


In [8]:
not_parsed_XML_files

[]